In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": "508e9465-523d-4c41-9070-0bbca38e1bcb",
       "fs.azure.account.oauth2.client.secret": "jJZ.N-HQgm-FIXsrFG4WK9.710XYT9Y6tu",
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/5f3f6c08-b8f8-4ea6-9622-f4df92fc75dd/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://files@storagesam.dfs.core.windows.net/",
mount_point = "/mnt/cdata",
extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-45419871445892> in <module> 
 9 source = "abfss://files@storagesam.dfs.core.windows.net/" , 
 10 mount_point = "/mnt/cdata" , 
 ---> 11 extra_configs = configs)
 
 /local_disk0/tmp/1616492289725-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling o244.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/cdata; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/cdata
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:470)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/cdata
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:404)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:670)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:504)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:659)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:412)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:89)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:290)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:250)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:434)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging

In [1]:
import pandas as pd
import openpyxl as xl
from datetime import datetime

In [2]:
def rdata(file_name):
  wb = xl.load_workbook(""+file_name)
  if 'Cost Units' in file_name:
    ws = wb['All Reports 2']
    data = list(ws.values)
    return data
  elif 'CR' in file_name:
    ws = wb['ASR20.Data CR']
    data = list(ws.values)
    return data
  elif 'RES' in file_name:
    ws = wb['ASR20.Data RES']
    data = list(ws.values)
    return data
  else:
    return 'File not accepted!'

In [3]:
file_name = "ASR20.Data Cost Units.xlsx"
data = rdata(file_name)
asset_line=6
df = pd.DataFrame(data[asset_line+3:asset_line+41])
df = df.T
row1 = df.iloc[0]
row2 = df.iloc[1]
header = [row1[0]]
for i in range(1,len(row1)):
  header.append(row1[i].replace(" ","_")+"_"+row2[i])

df = df.iloc[2:]
df.columns = header
df.reset_index(drop=True,inplace=True)
df.dropna(how='all',inplace=True)
title = data[asset_line-1][0].strip()
titles = [title]*len(df)
asset = data[asset_line][0].strip()
asset = asset[:len(asset)-17].strip("\n||\t")
assets = [asset]*len(df)
version = [file_name.split(".")[0].strip()]*len(df)
df.insert(0,'Title',title)
df.insert(1,'Asset_Name',asset)
df.insert(2,'Version',version)
# df.head()
tasr_cls_name = file_name.split(".")[1]
asr_cls_name = ""
if tasr_cls_name == 'Data Cost Units':
  asr_cls_name = 'n/a'
elif tasr_cls_name == 'Data RES':
  asr_cls_name = 'RES'
elif tasr_cls_name == 'Data CR':
  asr_cls_name = 'CR'
else:
  print('ASR Class not known.')

asr_cls = [asr_cls_name]*len(df)
cse = ""
if asr_cls_name=="RES" and title=='Proved Developed':
  cse = 'LOW'
elif asr_cls_name=='RES' and title=='Proved Undeveloped':
  cse = 'LOW'
elif asr_cls_name=='RES' and title=='Proved + Prob. Developed':
  cse = 'BASE'
elif asr_cls_name=='RES' and title=='Proved + Prob. Undeveloped':
  cse = 'BASE'
elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Developed +':
  cse = 'HIGH'
elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Undeveloped +':
  cse = 'HIGH'
elif asr_cls_name=="CR" and title=='Total Proved':
  cse = 'LOW'
elif asr_cls_name=="CR" and title=='Total Proved + Probable':
  cse = 'BASE'
elif asr_cls_name=="CR" and title=='Total Proved + Prob. + Poss.':
  cse = 'HIGH'
elif asr_cls_name=="n/a" and title=='Total Proved':
  cse = 'LOW'
elif asr_cls_name=="n/a" and title=='Total Proved + Probable':
  cse = 'BASE'
elif asr_cls_name=="n/a" and title=='Total Proved + Prob. + Poss.':
  cse = 'HIGH'
else:
  print("Invalid case!")

asr_cse = ""
if asr_cls_name == 'RES' and cse == 'LOW':
  asr_cse = '1P'
elif asr_cls_name == 'RES' and cse == 'BASE':
  asr_cse = '2P'
elif asr_cls_name == 'RES' and cse == 'HIGH':
  asr_cse = '3P'
elif asr_cls_name == 'CR' and cse == 'LOW':
  asr_cse = '1C'
elif asr_cls_name == 'CR' and cse == 'BASE':
  asr_cse = '2C'
elif asr_cls_name == 'CR' and cse == 'HIGH':
  asr_cse = '3C'
elif asr_cls_name == 'n/a' and cse == 'LOW':
  asr_cse = 'n/a'
elif asr_cls_name == 'n/a' and cse == 'BASE':
  asr_cse = 'n/a'
elif asr_cls_name == 'n/a' and cse == 'HIGH':
  asr_cse = 'n/a'
else:
  print('Invalid ASR Case!')

asr_cases = [asr_cse]*len(df)


bp_cases = [cse]*len(df)
df.insert(3,'ASR_Class',asr_cls)
df.insert(4,'BP_Case',bp_cases)
df.insert(5,'ASR_Case',asr_cases)

df_ri = df[df['Date'].isnull()].index

# df.head()
df.drop(df_ri,inplace=True)
df
# sdf = spark.createDataFrame(df)
# sdf.write.mode('overwrite').saveAsTable("DC")

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-01-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-6950.31682,0
1,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-02-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
2,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-03-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
3,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-04-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
4,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-05-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
5,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-06-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
6,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-07-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
7,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-08-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
8,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-09-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None
9,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-10-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.133,None


In [8]:
dfs1 = [df]

In [9]:
file_name = "ASR20.Data CR.xlsx"
data = rdata(file_name)
asset_line=6
df = pd.DataFrame(data[asset_line+3:asset_line+41])
df = df.T
row1 = df.iloc[0]
row2 = df.iloc[1]
header = [row1[0]]
for i in range(1,len(row1)):
  header.append(row1[i].replace(" ","_")+"_"+row2[i])

df = df.iloc[2:]
df.columns = header
df.reset_index(drop=True,inplace=True)
df.dropna(how='all',inplace=True)
title = data[asset_line-1][0].strip()
titles = [title]*len(df)
asset = data[asset_line][0].strip()
asset = asset[:len(asset)-17].strip("\n||\t")
assets = [asset]*len(df)
version = [file_name.split(".")[0].strip()]*len(df)
df.insert(0,'Title',title)
df.insert(1,'Asset_Name',asset)
df.insert(2,'Version',version)
# df.head()
tasr_cls_name = file_name.split(".")[1]
asr_cls_name = ""
if tasr_cls_name == 'Data Cost Units':
  asr_cls_name = 'n/a'
elif tasr_cls_name == 'Data RES':
  asr_cls_name = 'RES'
elif tasr_cls_name == 'Data CR':
  asr_cls_name = 'CR'
else:
  print('ASR Class not known.')

asr_cls = [asr_cls_name]*len(df)
cse = ""
if asr_cls_name=="RES" and title=='Proved Developed':
  cse = 'LOW'
elif asr_cls_name=='RES' and title=='Proved Undeveloped':
  cse = 'LOW'
elif asr_cls_name=='RES' and title=='Proved + Prob. Developed':
  cse = 'BASE'
elif asr_cls_name=='RES' and title=='Proved + Prob. Undeveloped':
  cse = 'BASE'
elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Developed +':
  cse = 'HIGH'
elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Undeveloped +':
  cse = 'HIGH'
elif asr_cls_name=="CR" and title=='Total Proved':
  cse = 'LOW'
elif asr_cls_name=="CR" and title=='Total Proved + Probable':
  cse = 'BASE'
elif asr_cls_name=="CR" and title=='Total Proved + Prob. + Poss.':
  cse = 'HIGH'
elif asr_cls_name=="n/a" and title=='Total Proved':
  cse = 'LOW'
elif asr_cls_name=="n/a" and title=='Total Proved + Probable':
  cse = 'BASE'
elif asr_cls_name=="n/a" and title=='Total Proved + Prob. + Poss.':
  cse = 'HIGH'
else:
  print("Invalid case!")

asr_cse = ""
if asr_cls_name == 'RES' and cse == 'LOW':
  asr_cse = '1P'
elif asr_cls_name == 'RES' and cse == 'BASE':
  asr_cse = '2P'
elif asr_cls_name == 'RES' and cse == 'HIGH':
  asr_cse = '3P'
elif asr_cls_name == 'CR' and cse == 'LOW':
  asr_cse = '1C'
elif asr_cls_name == 'CR' and cse == 'BASE':
  asr_cse = '2C'
elif asr_cls_name == 'CR' and cse == 'HIGH':
  asr_cse = '3C'
elif asr_cls_name == 'n/a' and cse == 'LOW':
  asr_cse = 'n/a'
elif asr_cls_name == 'n/a' and cse == 'BASE':
  asr_cse = 'n/a'
elif asr_cls_name == 'n/a' and cse == 'HIGH':
  asr_cse = 'n/a'
else:
  print('Invalid ASR Case!')

asr_cases = [asr_cse]*len(df)


bp_cases = [cse]*len(df)
df.insert(3,'ASR_Class',asr_cls)
df.insert(4,'BP_Case',bp_cases)
df.insert(5,'ASR_Case',asr_cases)

df_ri = df[df['Date'].isnull()].index

# df.head()
df.drop(df_ri,inplace=True)
df
# sdf = spark.createDataFrame(df)
# sdf.write.mode('overwrite').saveAsTable("DC")

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-01-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,0
1,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-02-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
2,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-03-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
3,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-04-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
4,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-05-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2033-01-01 00:00:00,0,143.074875,32836.856589,...,0,0,0,24148.516906,0,0,0,0,184706.099124,72772.353159
68,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2034-01-01 00:00:00,0,143.074875,32836.856589,...,0,0,0,24131.587295,0,0,0,0,188900.121055,95164.957695
69,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2035-01-01 00:00:00,0,117.225143,26904.131268,...,0,0,0,24614.219041,0,0,0,0,153419.317475,76117.764261
70,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2036-01-01 00:00:00,0,14.39833,3304.534819,...,0,0,0,5547.918536,0,41008.749315,0,0,-24252.123808,59.792373


In [10]:
dfs1.append(df)
fd = pd.concat(dfs1,ignore_index=True)
fd

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-01-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,0
1,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-02-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
2,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-03-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
3,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-04-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
4,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2021-05-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2033-01-01 00:00:00,0,143.074875,32836.856589,...,0,0,0,24148.516906,0,0,0,0,184706.099124,72772.353159
140,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2034-01-01 00:00:00,0,143.074875,32836.856589,...,0,0,0,24131.587295,0,0,0,0,188900.121055,95164.957695
141,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2035-01-01 00:00:00,0,117.225143,26904.131268,...,0,0,0,24614.219041,0,0,0,0,153419.317475,76117.764261
142,Total Proved,Petrel R1 Rese,ASR20,CR,LOW,1C,2036-01-01 00:00:00,0,14.39833,3304.534819,...,0,0,0,5547.918536,0,41008.749315,0,0,-24252.123808,59.792373


In [12]:
gd = pd.read_excel('ASR20.General Data.xlsx',sheet_name='ASR20.General Data')
asset_list = gd['Asset'].unique()[1:]
asset_list

array(['Gudrun', 'Ivar Aasen', 'Brage', '32nd round Kelvin', 'Fram',
       'Fram H-North', 'Byrding', 'Vega', 'Gjoa', 'Cygnus', 'Njord',
       'Hyme', 'Snohvit', 'Adorf Carbon', 'Adorf Scheerhorn', 'WEK',
       'AESW', 'Allermh Reitbrk W', 'Altmark', 'Annaveen Malm Oil',
       'Annaveen T', 'Apeldorn', 'Ruehlermoor', 'Ashrafi', 'Bauge',
       'Bentheim / Ochtrup', 'Boetersen Sum', 'Huemmling',
       'Boetersen EMPG', 'Boomerang', 'Bramberge', 'Boetersen Sued',
       'Bue', 'Roemerberg', 'Idsingen Walsrode', 'D15-A',
       'Sillimanite (UK)', 'D15-5', 'Draugen', 'Dugong', 'Duva', 'E11-1',
       'E17-3', 'E17a-A', 'Echino South', 'F16-A',
       'F17a-Rembrandt-Vermeer Oil', 'F3-B', 'F3-B LG Improved Recovery',
       'Fehndorf', 'Fenja', 'Thueringen', 'Sinstorf', 'G17d-AP',
       'Georgsdorf', 'Georgsdorf Infills 1', 'Georgsdorf Infills 2',
       'Grosbeak', 'Hamwiede K', 'Hemsbuende Sum', 'Heringsdorf', 'Husum',
       'Isabella', 'Jangkrik', 'Jangkrik Huile', 'Jangkrik Infi

In [27]:
def data_extract(file_name):
  sd = 0
  fd = 0
  data = rdata(file_name)
  asset_line=6
  ldf = []
  with open('log.txt','a') as f:
    dts = datetime.now()
    f.write(f"Processing '{file_name}' {dts.strftime('on %B %d,%Y at %X')} ...\n")
  try:
    while True:
      df = pd.DataFrame(data[asset_line+3:asset_line+41])
      df = df.T
      row1 = df.iloc[0]
      row2 = df.iloc[1]
      header = [row1[0]]
      for i in range(1,len(row1)):
        header.append(row1[i].replace(" ","_")+"_"+row2[i])

      df = df.iloc[2:]
      df.columns = header
      df.reset_index(drop=True,inplace=True)
      df.dropna(how='all',inplace=True)
      df_ri = df[df['Date'].isnull()].index
      df.drop(df_ri,inplace=True)
      tasr_cls_name = file_name.split(".")[1]
      asr_cls_name = ""
      if tasr_cls_name == 'Data Cost Units':
        asr_cls_name = 'n/a'
      elif tasr_cls_name == 'Data RES':
        asr_cls_name = 'RES'
      elif tasr_cls_name == 'Data CR':
        asr_cls_name = 'CR'
      else:
        print('ASR Class not known.')
      title = data[asset_line-1][0].strip()
      titles = [title]*len(df)
      asset = data[asset_line][0].strip()
      for ast in asset_list:
            if ast in asset:
                asset = ast
                break
      assets = [asset]*len(df)
      version = [file_name.split(".")[0].strip()]*len(df)
      df.insert(0,'Title',title)
      df.insert(1,'Asset_Name',assets)
      df.insert(2,'Version',version)
      # df.head()

      asr_cls = [asr_cls_name]*len(df)
      cse = ""
      if asr_cls_name=="RES" and title=='Proved Developed':
        cse = 'LOW'
      elif asr_cls_name=='RES' and title=='Proved Undeveloped':
        cse = 'LOW'
      elif asr_cls_name=='RES' and title=='Proved + Prob. Developed':
        cse = 'BASE'
      elif asr_cls_name=='RES' and title=='Proved + Prob. Undeveloped':
        cse = 'BASE'
      elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Developed':
        cse = 'HIGH'
      elif asr_cls_name=='RES' and title=='Proved + Prob. + Poss. Undeveloped':
        cse = 'HIGH'
      elif asr_cls_name=="CR" and title=='Total Proved':
        cse = 'LOW'
      elif asr_cls_name=="CR" and title=='Total Proved + Probable':
        cse = 'BASE'
      elif asr_cls_name=="CR" and title=='Total Proved + Prob. + Poss.':
        cse = 'HIGH'
      elif asr_cls_name=="n/a" and title=='Total Proved':
        cse = 'LOW'
      elif asr_cls_name=="n/a" and title=='Total Proved + Probable':
        cse = 'BASE'
      elif asr_cls_name=="n/a" and title=='Total Proved + Prob. + Poss.':
        cse = 'HIGH'
      else:
        print("Invalid case!")

      asr_cse = ""
      if asr_cls_name == 'RES' and cse == 'LOW':
        asr_cse = '1P'
      elif asr_cls_name == 'RES' and cse == 'BASE':
        asr_cse = '2P'
      elif asr_cls_name == 'RES' and cse == 'HIGH':
        asr_cse = '3P'
      elif asr_cls_name == 'CR' and cse == 'LOW':
        asr_cse = '1C'
      elif asr_cls_name == 'CR' and cse == 'BASE':
        asr_cse = '2C'
      elif asr_cls_name == 'CR' and cse == 'HIGH':
        asr_cse = '3C'
      elif asr_cls_name == 'n/a' and cse == 'LOW':
        asr_cse = 'n/a'
      elif asr_cls_name == 'n/a' and cse == 'BASE':
        asr_cse = 'n/a'
      elif asr_cls_name == 'n/a' and cse == 'HIGH':
        asr_cse = 'n/a'
      else:
        print('Invalid ASR Case!')

      asr_cases = [asr_cse]*len(df)
      bp_cases = [cse]*len(df)
      df.insert(3,'ASR_Class',asr_cls)
      df.insert(4,'BP_Case',bp_cases)
      df.insert(5,'ASR_Case',asr_cases)
      asset_line+=49
      with open('log.txt','a') as f:
        if df.empty:
            fd+=1
            f.write(f"'{asset}' in '{file_name}' have no data\n")
        sd+=1
      ldf.append(df)
  except IndexError as e:
    pass
  finally:
    with open('log.txt','a') as f:
        f.write(f"Total number of assets :{sd+fd}.\n")
        f.write(f"Number of assets which have data :{sd}.\n")
        f.write(f"Number of assets which have no data :{fd}.\n")
    return pd.concat(ldf,ignore_index=True)


In [28]:
file_name = "ASR20.Data Cost Units.xlsx"
file_name1 = "ASR20.Data CR.xlsx"
file_name2 = "ASR20.Data RES.xlsx"
fdf_dcu = data_extract(file_name)
fdf_cr = data_extract(file_name1)
fdf_res = data_extract(file_name2)

In [30]:
fdf_dcu.head()

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-01-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-6950.31682,0
1,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-02-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.13300,None
2,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-03-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.13300,None
3,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-04-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.13300,None
4,Total Proved,Germany Adjustment,ASR20,n/a,LOW,n/a,2021-05-01 00:00:00,0,0,0,...,0,0,0,0,0,0,0,0,-359.13300,None


In [25]:
fdf_cr

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Total Proved,Petrel R1,ASR20,CR,LOW,1C,2021-01-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287000,0
1,Total Proved,Petrel R1,ASR20,CR,LOW,1C,2021-02-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287000,None
2,Total Proved,Petrel R1,ASR20,CR,LOW,1C,2021-03-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287000,None
3,Total Proved,Petrel R1,ASR20,CR,LOW,1C,2021-04-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287000,None
4,Total Proved,Petrel R1,ASR20,CR,LOW,1C,2021-05-01 00:00:00,0,0,0,...,0,0,0,190.287,0,0,0,0,-190.287000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Total Proved,Boetersen EMPG,ASR20,CR,LOW,1C,2039-01-01 00:00:00,0,0,72.075295,...,0,0,0,0,40.469342,0,0,0,634.019689,183.865710
287,Total Proved,Boetersen EMPG,ASR20,CR,LOW,1C,2040-01-01 00:00:00,0,0,57.470782,...,0,0,0,0,32.914479,0,0,0,515.660173,149.541450
288,Total Proved,Boetersen EMPG,ASR20,CR,LOW,1C,2041-01-01 00:00:00,0,0,45.527757,...,0,0,0,0,26.595999,0,0,0,416.670653,120.834489
289,Total Proved,Boetersen EMPG,ASR20,CR,LOW,1C,2042-01-01 00:00:00,0,0,24.987580,...,0,0,0,0,14.888962,0,0,0,233.260404,67.645517


In [31]:
fdf_res

,Title,Asset_Name,Version,ASR_Class,BP_Case,ASR_Case,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,...,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
0,Proved Developed,Touat,ASR20,RES,LOW,1P,2021-01-01 00:00:00,0,0,632.349337,...,0,0,0,2262.511347,1395.070844,0,0,0,-17237.273036,7907.445303
1,Proved Developed,Touat,ASR20,RES,LOW,1P,2021-02-01 00:00:00,0,0,583.866325,...,0,0,0,2262.511347,1395.070844,0,0,0,-1182.873382,None
2,Proved Developed,Touat,ASR20,RES,LOW,1P,2021-03-01 00:00:00,0,0,427.754567,...,0,0,0,2262.511347,1395.070844,0,0,0,-1851.734210,None
3,Proved Developed,Touat,ASR20,RES,LOW,1P,2021-04-01 00:00:00,0,10.863215,2196.115839,...,0,0,0,2262.511347,1305.319263,0,0,0,6047.513421,None
4,Proved Developed,Touat,ASR20,RES,LOW,1P,2021-05-01 00:00:00,0,11.201220,2253.524714,...,0,0,0,2262.511347,1305.319263,0,0,0,6299.368950,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,Proved + Prob. + Poss. Undeveloped,Seagull,ASR20,RES,HIGH,3P,2029-01-01 00:00:00,1473.377538,18.030304,1803.030364,...,0,0,0,37693.053192,0,0,0,0,83883.517070,0
15726,Proved + Prob. + Poss. Undeveloped,Seagull,ASR20,RES,HIGH,3P,2030-01-01 00:00:00,1317.290273,16.992923,1699.292254,...,0,0,0,42568.048654,0,0,0,0,69486.399683,0
15727,Proved + Prob. + Poss. Undeveloped,Seagull,ASR20,RES,HIGH,3P,2031-01-01 00:00:00,1180.029383,18.478668,1847.866753,...,0,0,0,44128.777541,0,0,0,0,62761.820732,0
15728,Proved + Prob. + Poss. Undeveloped,Seagull,ASR20,RES,HIGH,3P,2032-01-01 00:00:00,994.027428,17.652617,1765.261717,...,0,0,0,44619.392881,0,0,0,0,50167.628330,0


In [21]:
fdf_dcu.to_csv("Data Cost Units.csv",index=False)
fdf_cr.to_csv("Data CR.csv",index=False)
fdf_res.to_csv("Data RES.csv",index=False)

In [22]:
final_df = pd.concat([fdf_dcu,fdf_cr,fdf_res])
final_df.to_csv("Sample Master.csv",index=False)

In [ ]:
%sql
select * from DC

Title,Asset_Name,Version,ASR_Class,Cases,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,NGL_Mbbl,Oil_Equivalent_MBOE,Revenue_Oil_M$US,Revenue_Condensate_M$US,Revenue_Gas_M$US,Revenue_NGL_M$US,Total_Revenue_M$US,Capex_Explo_Site_Survey_M$US,Capex_Explo_Drilling_-_Dry_Hole_M$US,Capex_Explo_Drilling_-_Success_M$US,Capex_Explo_Appraisal_Drilling_M$US,Development_Drilling_M$US,Capex_Workover_M$US,Plant_&_Euipment_M$US,Maintenance_Total_M$US,Capex_Total_M$US,Opex_Fixed_M$US,Opex_Workover_M$US,Opex_Sharing_M$US,Opex_Variable_M$US,Opex_Transportation_M$US,Opex_Treatment_M$US,Opex_Total_M$US,Epex_G&G_M$US,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-01-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.656488356922,139.656488356922,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-139.656488356922,0
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-02-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.887142401785,139.887142401785,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-139.887142401785,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-03-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.118177390554,140.118177390554,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.118177390554,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-04-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.349593952392,140.349593952392,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.349593952392,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-05-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.581392717496,140.581392717496,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.581392717496,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-06-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.813574317106,140.813574317106,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.813574317106,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-07-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.046139383504,141.046139383504,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.046139383504,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-08-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.279088550018,141.279088550018,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.279088550018,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-09-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.51242245102,141.51242245102,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.51242245102,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-10-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.746141721929,141.746141721929,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.746141721929,null


In [36]:
gd = pd.read_excel("ASR20.General Data.xlsx",sheet_name="ASR20.General Data")
lgd = list(gd.columns)
lgd = ['Entity_Name']+lgd[1:]
gd.columns = lgd
gd = gd.iloc[1:,:len(gd.columns)-21]
gd.reset_index(drop=True,inplace=True)
gd

,Entity_Name,Flagged,Well Name,Product List,Well Type,Well Profile Type,Oil Reserves Type,Gas Reserves Type,Forecast Mode,Entity Currency,...,Interdependencies,License #,License Expiry,License Type,Neptune - CGU,NL filter,POSe,POSg,Project Maturity,Region
0,15/3-A-13,0.0,15/3-A-13,Oil,Oil,NaN,Light and Medium Oil,Gas,Total,Default (Norwegian Kroner),...,NaN,PL 025,2028-12-31,Concession,NO Gudrun,NaN,NaN,NaN,NaN,North Sea
1,15/3-A-14,0.0,15/3-A-14,Oil,Oil,NaN,Light and Medium Oil,Gas,Total,Default (Norwegian Kroner),...,NaN,PL 025,2028-12-31,Concession,NO Gudrun,NaN,NaN,NaN,NaN,North Sea
2,15/3-A-15,1.0,15/3-A-15,Gas,Oil,NaN,Light and Medium Oil,Gas,Total,Default (Norwegian Kroner),...,NaN,NaN,2028-12-31,Concession,NO Gudrun,NaN,NaN,NaN,NaN,North Sea
3,15/3-A-16,0.0,15/3-A-16,Gas,Gas,NaN,Light and Medium Oil,Gas,Total,Default (Norwegian Kroner),...,NaN,PL 025,2028-12-31,Concession,NO Gudrun,NaN,NaN,NaN,NaN,North Sea
4,15/3-A-5 A,0.0,15/3-A-5,Oil,Oil,NaN,Light and Medium Oil,Gas,Total,Default (Norwegian Kroner),...,NaN,PL 025,2028-12-31,Concession,NO Gudrun,NaN,NaN,NaN,NaN,North Sea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,PY-H Z1/1,0.0,692:0154100,Gas,Gas,NaN,Light and Medium Oil,Gas,Total,Default (Euro),...,NaN,NaN,NaT,NaN,GE Gas West,NaN,NaN,NaN,NaN,Rotenburg-Taaken
931,PY-H Z1/2,0.0,692:0154106,Gas,Gas,NaN,Light and Medium Oil,Gas,Total,Default (Euro),...,NaN,NaN,NaT,NaN,GE Gas West,NaN,NaN,NaN,NaN,Rotenburg-Taaken
932,Q13-8,0.0,Q13-8,Gas,Cost Entity,NaN,Light and Medium Oil,Gas,Total,Default (Euro),...,NaN,K12/K15,2023-02-18,Concession,NaN,NaN,NaN,NaN,NaN,NaN
933,Rita Cost Entity,0.0,Rita Cost Entity,Gas,Cost Entity,NaN,Light and Medium Oil,Gas,Total,Default (British Pound),...,NaN,P766 & P.771,2027-06-13,Concession,UK Other,NaN,NaN,NaN,NaN,NaN


In [ ]:
%sql 
select count(*) from DC

count(1)
49


In [ ]:
%sql 
create or replace view v1 as (select * from DC)

In [ ]:
%sql 
select * from v1

Title,Asset_Name,Version,ASR_Class,Cases,Date,Oil_Mbbl,Condensate_Mbbl,Gas_MMcf,NGL_Mbbl,Oil_Equivalent_MBOE,Revenue_Oil_M$US,Revenue_Condensate_M$US,Revenue_Gas_M$US,Revenue_NGL_M$US,Total_Revenue_M$US,Capex_Explo_Site_Survey_M$US,Capex_Explo_Drilling_-_Dry_Hole_M$US,Capex_Explo_Drilling_-_Success_M$US,Capex_Explo_Appraisal_Drilling_M$US,Development_Drilling_M$US,Capex_Workover_M$US,Plant_&_Euipment_M$US,Maintenance_Total_M$US,Capex_Total_M$US,Opex_Fixed_M$US,Opex_Workover_M$US,Opex_Sharing_M$US,Opex_Variable_M$US,Opex_Transportation_M$US,Opex_Treatment_M$US,Opex_Total_M$US,Epex_G&G_M$US,Epex_Seismic_M$US,Epex_G&A_M$US,Epex_Area_Fee_M$US,Opex_+_Epex_M$US,Royalty_M$US,Abandonment_M$US,Revenue_Processing_M$US,Revenue_Pipeline_M$US,Before_Tax_Cash_Flow_M$US,Tax_Paid_M$US
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-01-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.656488356922,139.656488356922,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-139.656488356922,0
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-02-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.887142401785,139.887142401785,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-139.887142401785,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-03-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.118177390554,140.118177390554,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.118177390554,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-04-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.349593952392,140.349593952392,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.349593952392,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-05-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.581392717496,140.581392717496,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.581392717496,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-06-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.813574317106,140.813574317106,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-140.813574317106,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-07-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.046139383504,141.046139383504,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.046139383504,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-08-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.279088550018,141.279088550018,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.279088550018,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-09-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.51242245102,141.51242245102,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.51242245102,null
Total Proved,Germany Adjustment,ASR20,n/a,Total Proved,2024-10-01T00:00:00.000+0000,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.746141721929,141.746141721929,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0,-141.746141721929,null


In [ ]:
%sql 
update v1 
set Cases = case when Cases = 'Total Proved' then 'Low' end

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation[Title#2634,Asset_Name#2635,Version#2636,ASR_Class#2637,Cases#2638,Date#2639,Oil_Mbbl#2640L,Condensate_Mbbl#2641L,Gas_MMcf#2642L,NGL_Mbbl#2643L,Oil_Equivalent_MBOE#2644L,Revenue_Oil_M$US#2645L,Revenue_Condensate_M$US#2646L,Revenue_Gas_M$US#2647L,Revenue_NGL_M$US#2648L,Total_Revenue_M$US#2649,Capex_Explo_Site_Survey_M$US#2650,Capex_Explo_Drilling_-_Dry_Hole_M$US#2651,Capex_Explo_Drilling_-_Success_M$US#2652,Capex_Explo_Appraisal_Drilling_M$US#2653,Development_Drilling_M$US#2654,Capex_Workover_M$US#2655,Plant_&_Euipment_M$US#2656,Maintenance_Total_M$US#2657,... 19 more fields] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:343)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:211)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:65)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:80)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:216)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:65)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:58)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:152)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:89)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:149)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:141)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:141)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:224)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:218)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:181)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:202)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:201)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:132)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.QueryExecuti

In [34]:
for i in range(10):
    print(i)
    if i == 4:
        break

0
1
2
3
4
